In [1]:
import os

In [2]:
%pwd

'/Users/suyash/Desktop/End to End/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/suyash/Desktop/End to End'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.IPL_Predictor.constants import *
from src.IPL_Predictor.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
from src.IPL_Predictor import logger
import zipfile

In [9]:
## component-Data Ingestion

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-24 00:51:26,977: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-24 00:51:26,979: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-24 00:51:26,982: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-24 00:51:26,982: INFO: common: created directory at : artifacts]
[2025-09-24 00:51:26,983: INFO: common: created directory at : artifacts/data_ingestion]
[2025-09-24 00:51:28,678: INFO: 251335794: artifacts/data_ingestion/IPL_Prediction%20file.zip download! with following info: 
Connection: close
Content-Length: 2840372
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "0957b239deaca226251c7967a5ae532ee13f49760907051a97fd462a43b0cd0c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: CF25:36247:1CACED:401CD3:68D2F336
Accept-Ranges: 